In [6]:
# #@markdown Check type of GPU and VRAM available.
# !nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [7]:
# save_to_gdrive = True #@param {type:"boolean"}
# if save_to_gdrive:
#     from google.colab import drive
#     drive.mount('/content/drive')

In [8]:
import os
import json
import pickle
import numpy as np
import pandas as pd

In [9]:
#autocast_questions = json.load(open('autocast_questions.json')) # from the Autocast dataset
test_questions = json.load(open('auxillary-data/autocast_competition_test_set.json'))
test_ids = [q['id'] for q in test_questions]
test_questions_df = pd.DataFrame(test_questions)

In [10]:
autocast_questions = json.load(open('auxillary-data/autocast_train_questions.json'))

autocast_questions_df = pd.DataFrame(autocast_questions)

In [11]:
autocast_questions_df.head()
print(len(autocast_questions_df))

6532


In [12]:
autocast_questions_df.head()['question'][0]

'What will the end-of-day closing value for the dollar against the renminbi be on 1 January 2016?'

In [13]:
autocast_questions_df[(autocast_questions_df['id']=='G1') | (autocast_questions_df['id']=='G4') | (autocast_questions_df['id']=='M363')][['id', 'qtype', 'question', 'choices', 'answer', 'tags']]

# G1 G4 M363

,id,qtype,question,choices,answer,tags
0,G1,mc,What will the end-of-day closing value for the...,"[Less than 6.30, Between 6.30 and 6.35, inclus...",D,"[Finance, Economic Indicators]"
2,G4,t/f,Will there be an initial public offering on ei...,"[yes, no]",yes,[Finance]
2291,M363,num,When will the first 100 Megawatt fusion-based ...,None,None,"[Technological Advances, Technology – Energy, ..."


In [14]:
autocast_questions_df[(autocast_questions_df['qtype'] == 'num') & (autocast_questions_df['answer'].isna() == False)]

,question,id,background,publish_time,close_time,tags,source_links,prediction_count,forecaster_count,answer,choices,status,qtype,crowd
2320,How many Federally Insured Commercial Banking ...,M402,The U.S. has seen a continued consolidation of...,2016-12-31 00:01:25+00:00,2017-06-01 04:59:00+00:00,[Business],"[https://www5.fdic.gov/hsob/HSOBRpt.asp, https...",97,30,0.42049,"{'max': 5500, 'min': 4500, 'deriv_ratio': 1}",Resolved,num,[{'timestamp': '2016-12-31 00:02:47.669752+00:...
2321,In what month and year will an AI system score...,M403,Starcraft 2 (SC2) is a real time strategy comp...,2016-12-31 00:09:27+00:00,2019-02-05 05:01:00+00:00,"[Computing – Artificial Intelligence, Computin...","[http://us.battle.net/sc2/en/, http://wiki.tea...",514,258,0.376395,"{'max': '2030-01-01', 'min': '2017-02-01', 'de...",Resolved,num,[{'timestamp': '2016-12-31 00:10:20.723851+00:...
2323,How long until a machine-learning system can t...,M405,"Since the early days of ""machine code"" program...",2016-12-31 00:23:03+00:00,2020-01-01 00:00:00+00:00,"[Computing – Artificial Intelligence, Computin...","[https://openreview.net/pdf?id=ByldLrqlx, http...",263,169,0.408719,"{'max': '2050-01-01', 'min': '2017-02-01', 'de...",Resolved,num,[{'timestamp': '2016-12-31 00:27:37.814118+00:...
2331,When will the first exaflop performer appear?,M415,Supercomputers just keep getting more capable....,2017-01-05 05:13:34+00:00,2018-12-31 12:00:00+00:00,[Computing – Computers],"[https://www.metaculus.com/www.top500.org, htt...",147,80,0.849452,"{'max': '2025-01-01', 'min': '2017-01-15', 'de...",Resolved,num,[{'timestamp': '2017-01-05 05:33:45.045160+00:...
2333,When will the first law concerning artificial ...,M418,"As of now, there are to the author's knowledge...",2017-01-09 17:54:54+00:00,2021-02-01 15:29:00+00:00,"[Computing – Artificial Intelligence, Law – Le...",[https://www.gpo.gov/],422,201,0.260335,"{'max': '2060-01-01', 'min': '2017-02-01', 'de...",Resolved,num,[{'timestamp': '2017-01-09 17:56:35.417775+00:...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,What will be the 7-day moving average of daily...,M10818,Following the decline of Omicron wave in Virgi...,2022-04-29 16:00:00+00:00,2022-05-06 16:00:00+00:00,"[Novel Coronavirus (Covid-19), Tournament -- R...",[https://www.vdh.virginia.gov/coronavirus/see-...,50,18,0.168667,"{'max': 15000, 'min': 0, 'deriv_ratio': 1}",Resolved,num,[{'timestamp': '2022-04-29 20:19:58.808115+00:...
6403,What will be the 7-day moving average of daily...,M10819,Following the decline of Omicron wave in Virgi...,2022-04-29 16:00:00+00:00,2022-05-20 04:00:00+00:00,"[Novel Coronavirus (Covid-19), Tournament -- R...",[https://www.vdh.virginia.gov/coronavirus/see-...,62,19,0.214533,"{'max': 15000, 'min': 0, 'deriv_ratio': 1}",Resolved,num,[{'timestamp': '2022-04-29 19:22:18.325765+00:...
6404,What will be the 7-day moving average of curre...,M10820,"Nationwide, new COVID-19 hospitalizations have...",2022-04-29 16:00:00+00:00,2022-05-06 16:00:00+00:00,"[Novel Coronavirus (Covid-19), Tournament -- R...",[https://covid.cdc.gov/covid-data-tracker/#new...,53,19,0.069975,"{'max': 4000, 'min': 0, 'deriv_ratio': 1}",Resolved,num,[{'timestamp': '2022-04-29 22:58:12.852895+00:...
6405,What will be the 7-day moving average of curre...,M10821,"Nationwide, new COVID-19 hospitalizations have...",2022-04-29 16:00:00+00:00,2022-05-20 16:00:00+00:00,"[Novel Coronavirus (Covid-19), Tournament -- R...",[https://covid.cdc.gov/covid-data-tracker/#new...,79,17,0.1211,"{'max': 4000, 'min': 0, 'deriv_ratio': 1}",Resolved,num,[{'timestamp': '2022-04-29 22:59:51.781406+00:...


In [15]:
autocast_questions_df[autocast_questions_df['id'] == 'G4']

,question,id,background,publish_time,close_time,tags,source_links,prediction_count,forecaster_count,answer,choices,status,qtype,crowd
2,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30.138000+00:00,2015-11-30 14:00:15+00:00,[Finance],[http://atimes.com/2015/11/china-will-allow-su...,545,148,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."


In [16]:
# Create a new dataframe with three columns
mcq_df = pd.DataFrame(columns=["question", "choices", "answer"])
true_false_df = pd.DataFrame(columns=["question", "choices", "answer"])

# Iterate over the autocast_questions_df dataframe
for index, row in autocast_questions_df.iterrows():
    if row['id'] in test_ids: # skipping questions in the competition test set
        # print(row['id'])
        continue
    if pd.isna(row['answer']): # skipping questions without answer
        continue
        
    if row['qtype'] == 'mc':
    # Extract the question and choices from the row
        
        question_mc = row["question"]
        id = row['id']
        background_mc = row['background']
        tags_mc = row['tags']
        choices_mc = row["choices"]
        answer_mc = row['answer']

        # Add a new row to the new dataframe
        new_row_mcq = {"question": question_mc, "id": id, "background": background_mc, "tags": tags_mc, "choices": choices_mc, "answer": answer_mc}
        mcq_df = pd.concat([mcq_df, pd.DataFrame([new_row_mcq])], ignore_index=True)
    
    # Extract the question and choices from the row
    elif row['qtype'] == 't/f':
        question_tf = row["question"]
        id = row['id']
        background_tf = row['background']
        tags_tf = row['tags']
        choices_tf = row["choices"]
        answer_tf = row['answer']

        # Add a new row to the new dataframe
        new_row_tf = {"question": question_tf, "id": id, "background": background_tf, "tags": tags_tf, "choices": choices_tf, "answer": answer_tf}
        true_false_df = pd.concat([true_false_df, pd.DataFrame([new_row_tf])], ignore_index=True)


In [17]:
mcq_df.head()
mcq_df.to_csv('auxillary-data/mcq-data.csv')

In [18]:
print(len(mcq_df))

744


In [19]:
print(autocast_questions_df.iloc[0]['question'])
print(autocast_questions_df.iloc[0]['id'])
print(autocast_questions_df.iloc[0]['choices'])
print(autocast_questions_df.iloc[0]['answer'])
print(autocast_questions_df.iloc[0]['background'])
print(autocast_questions_df.iloc[0]['tags'])

What will the end-of-day closing value for the dollar against the renminbi be on 1 January 2016?
G1
['Less than 6.30', 'Between 6.30 and 6.35, inclusive', 'More than 6.35 but less than 6.40', '6.40 or more']
D
Outcome will be determined by the end-of-day closing value reported by Bloomberg, at http://www.bloomberg.com/quote/usdcny:cur. For historical trends, see http://www.bloomberg.com/quote/usdcny:cur/chart. For more information on China's economy see http://www.theworldin.com/article/10492.
['Finance', 'Economic Indicators']


In [20]:
print(autocast_questions_df.iloc[2]['question'])
print(autocast_questions_df.iloc[2]['id'])
print(autocast_questions_df.iloc[2]['choices'])
print(autocast_questions_df.iloc[2]['answer'])
print(autocast_questions_df.iloc[2]['background'])
print(autocast_questions_df.iloc[2]['tags'])

Will there be an initial public offering on either the Shanghai Stock Exchange or the Shenzhen Stock Exchange before 1 January 2016?
G4
['yes', 'no']
yes
China suspended initial public offerings (IPOs) in early July (http://www. bloomberg. com/news/articles/2015-07-04/china-stock-brokers-set-up-19-billion-fund-to-stem-market-rout , http://www. reuters. com/article/2015/07/05/us-china-markets-brokerage-pledge-idUSKCN0PE08E20150705 , http://www. wsj. com/articles/china-setting-up-fund-to-stabilize-stock-market-1435991611 ). 
['Finance']


In [21]:
print(autocast_questions_df.iloc[2320]['question'])
print(autocast_questions_df.iloc[2320]['id'])
print(autocast_questions_df.iloc[2320]['choices'])
print(autocast_questions_df.iloc[2320]['answer'])
print(autocast_questions_df.iloc[2320]['background'])
print(autocast_questions_df.iloc[2320]['tags'])

How many Federally Insured Commercial Banking Institutions will there be at the end of 2017?
M402
{'max': 5500, 'min': 4500, 'deriv_ratio': 1}
0.42049
The U.S. has seen a continued consolidation of the Banking Industry over the past 70+ Years. According to the FDIC, there were over 14,000 FDIC Insured Commercial banks in 1934. Over the next 50 years, the Country maintained similar levels of Commercial Banks (i.e., over 14,000 FDIC Insured Commercial banks in 1984). However, over the past 30+ years, this number has shrunk rapidly through bank mergers and failures. As of 3rd quarter of 2016, year end, there are just 5,170 banks in the United States; the lowest year end total on record. Conversely, banks balance sheets have been growing rapidly over the same time horizon. According to the same FDIC report, domestic deposits at these banks as of Q3 2016 are at the highest levels ever, at over $10.5 trillion. The reasoning for this consolidation are wide ranging, including impacts from the 

In [22]:
for column in autocast_questions_df.columns:
    print(column + ': '+ str(type(autocast_questions_df.iloc[0][column]).__name__))

question: str
id: str
background: str
publish_time: str
close_time: str
tags: list
source_links: list
prediction_count: int
forecaster_count: int
answer: str
choices: list
status: str
qtype: str
crowd: list


In [23]:
true_false_df.head()
true_false_df.to_csv('auxillary-data/true-false.csv')

In [24]:
true_false_df.head()
true_false_df

,question,choices,answer,id,background,tags
0,Will there be an initial public offering on ei...,"[yes, no]",yes,G4,China suspended initial public offerings (IPOs...,[Finance]
1,Will the Export-Import Bank of the United Stat...,"[yes, no]",yes,G5,The Export-Import Bank's authorization expired...,"[Economic Policy, US Politics, US Policy]"
2,Will a trilateral meeting take place between C...,"[yes, no]",no,G6,"A trilateral meeting of leaders from China, Ja...",[Foreign Policy]
3,Will Iran release Jason Rezaian before 31 Octo...,"[yes, no]",yes,G7,For details of the case involving Jason Rezaia...,"[Foreign Policy, Security and Conflict]"
4,Will North Korea launch a land based missile w...,"[yes, no]",yes,G8,A launch for military or testing purposes woul...,[Security and Conflict]
...,...,...,...,...,...,...
1585,Will Apple release a new iMac based on Apple s...,"[yes, no]",yes,M6643,On June 22nd 2020 Apple announced transitionin...,"[Series — Slow Boring, Business, Computing – C..."
1586,"Will the Substack Slow Boring exceed 10,000 pa...","[yes, no]",yes,M6647,Substack is an online platform that provides p...,"[Series — Slow Boring, Business, Entertainment]"
1587,Will the US experience a 4th wave of COVID bef...,"[yes, no]",no,M6729,The US has to-date experience 3 waves of COVID...,[Novel Coronavirus (Covid-19)]
1588,Will Brazil have a 7-day rolling average above...,"[yes, no]",yes,M6777,"During the first wave in Brazil, the 7-day rol...",[Novel Coronavirus (Covid-19)]


In [25]:
true_false_df[true_false_df['id'] == 'G1']

,question,choices,answer,id,background,tags


In [26]:
print(len(true_false_df[true_false_df['answer'] == 'yes']))
print(len(true_false_df[true_false_df['answer'] == 'no']))
print(len(true_false_df))

450
1140
1590


In [27]:
autocast_questions_df.loc[1, 'question'] 

"How many seats will the Justice and Development Party (AKP) win in Turkey's snap elections?"

## Create baseline models outputting random answers

In [28]:
# import torch
# from transformers import GPT2Tokenizer, AutoModelForMultipleChoice

# model_name = 'gpt2'
# model = AutoModelForMultipleChoice.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [39]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained BERT model and tokenizer
model_name = 'bert-large-uncased'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [30]:
def random_baseline_model(question):
    if question['qtype'] == 't/f':
        return np.random.random(size=2)
    elif question['qtype'] == 'mc':
        probs = np.random.random(size=len(question['choices']))
        return probs / probs.sum()
    elif question['qtype'] == 'num':
        return np.random.random()

def caliberated_tf_pred(question):

    actual_question = question['question']
    choice1 = question['choices'][0]
    choice2 = question['choices'][1]

    input_text = [f"{actual_question} {choice1} {choice2}"]
    
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length= 256)

    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)

    return probs.detach().flatten().numpy()

def caliberated_mcq_pred(question):

    actual_question = question['question']
    choices = question['choices']

    input_text = [f"{actual_question} {choice}" for choice in choices]
    
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)

    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    
    choice_probs = probs[:, 1].detach().flatten().numpy()

    return choice_probs

def calibrated_random_baseline_model(question):
    if question['qtype'] == 't/f':
        # pred_idx = np.argmax(np.random.random(size=2))
        # pred = np.ones(2)
        # pred[pred_idx] += 1e-3
        # return pred / pred.sum()
        # return caliberated_tf_pred(question)
        return np.array([0.5, 0.5])
    elif question['qtype'] == 'mc':
        # pred_idx = np.argmax(np.random.random(size=len(question['choices'])))
        # pred = np.ones(len(question['choices']))
        # pred[pred_idx] += 1e-1
        # return pred / pred.sum()
        # return caliberated_mcq_pred(question)
        pred = np.ones(len(question['choices'])) / len(question['choices'])
        return pred

    elif question['qtype'] == 'num':
        return 0.415
        # return np.random.uniform(0.38, 0.41)

In [31]:
for index, row in autocast_questions_df.iterrows():
    question1 = row['question']
    choices = row['choices']

    print(question1)
    print(choices)
    break

What will the end-of-day closing value for the dollar against the renminbi be on 1 January 2016?
['Less than 6.30', 'Between 6.30 and 6.35, inclusive', 'More than 6.35 but less than 6.40', '6.40 or more']


## Get performance on the Autocast train set

Note that the Autocast dataset contains questions in the competition test set. Those should not be used.

In [32]:
def brier_score(probabilities, answer_probabilities):
    return ((probabilities - answer_probabilities) ** 2).sum() / 2

In [33]:
print(autocast_questions_df.iloc[1])

question            How many seats will the Justice and Developmen...
id                                                                 G2
background          The Justice and Development Party (AKP) failed...
publish_time                         2015-09-01 13:54:25.050000+00:00
close_time                                  2015-11-01 22:00:20+00:00
tags                       [Elections and Referenda, Non-US Politics]
source_links        [http://www.al-monitor.com/pulse/originals/201...
prediction_count                                                  567
forecaster_count                                                  194
answer                                                              A
choices                    [A majority, A plurality, Not a plurality]
status                                                       Resolved
qtype                                                              mc
crowd               [{'timestamp': '2015-09-01 00:00:00+00:00', 'f...
Name: 1, dtype: obje

In [34]:
preds = []
answers = []
qtypes = []
for question in autocast_questions:
    if question['id'] in test_ids: # skipping questions in the competition test set
        continue
    if question['answer'] is None: # skipping questions without answer
        continue
    preds.append(calibrated_random_baseline_model(question))
    if question['qtype'] == 't/f':
        ans_idx = 0 if question['answer'] == 'no' else 1
        # print(len(question['choices']))
        ans = np.zeros(len(question['choices']))
        ans[ans_idx] = 1
        qtypes.append('t/f')
    elif question['qtype'] == 'mc':
        ans_idx = ord(question['answer']) - ord('A')
        # print(len(question['choices']))
        ans = np.zeros(len(question['choices']))
        ans[ans_idx] = 1
        qtypes.append('mc')
    elif question['qtype'] == 'num':
        ans = float(question['answer'])
        qtypes.append('num')
    answers.append(ans)
    

## Evaluate the model

In [35]:
tf_results, mc_results, num_results = [],[],[]

for p, a, qtype in zip(preds, answers, qtypes):
    if qtype == 't/f':
        tf_results.append(brier_score(p, a))
    elif qtype == 'mc':
        mc_results.append(brier_score(p, a))
    else:
        num_results.append(np.abs(p - a))

print(f"T/F: {np.mean(tf_results)*100:.2f}, MCQ: {np.mean(mc_results)*100:.2f}, NUM: {np.mean(num_results)*100:.2f}")
print(f"Combined Metric: {(np.mean(tf_results) + np.mean(mc_results) + np.mean(num_results))*100:.2f}")

T/F: 25.00, MCQ: 38.05, NUM: 21.26
Combined Metric: 84.30


## Make predictions on test set

In [36]:
preds = []
for question in test_questions:
    preds.append(calibrated_random_baseline_model(question))

In [37]:
if not os.path.exists('submission'):
    os.makedirs('submission')

with open(os.path.join('submission', 'predictions.pkl'), 'wb') as f:
    pickle.dump(preds, f, protocol=2)

!cd submission && zip ../submission.zip ./* && cd ..
print("here")

  adding: predictions.pkl (deflated 79%)
here


In [38]:
!ls

Competition sign-up.xlsx        starter.ipynb
autocast_test_set_w_answers.csv submission
auxillary-data                  submission.zip
evaluation.ipynb
